<a href="https://colab.research.google.com/github/sugiyama404/ReinfoceLearningForTrading/blob/main/DuelingNetworkDQN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import random
from google.colab import drive
import copy

from datetime import datetime
from matplotlib import pyplot as plt
import pickle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, ReLU, Conv1D, Input, Lambda
from tensorflow.keras.optimizers import RMSprop, Adam
import tensorflow.keras as keras
from tensorflow.keras.losses import Huber
from tensorflow.keras import backend as K

optimizer = RMSprop()

from sklearn.preprocessing import StandardScaler


mode = 'test'
name = 'dn_dqn'

drive.mount('/content/drive/')
nov_dir = 'Colab Notebooks/dataset/reinforcement_learning/'
nov_path = '/content/drive/My Drive/' + nov_dir + f'sp500_{mode}.csv'

exp_dir = 'Colab Notebooks/workspace/export/'
mdl_dir = '/content/drive/My Drive/' + exp_dir + 'models'
csv_path = '/content/drive/My Drive/' + exp_dir + f'csv_data/{name}_{mode}.csv'

df = pd.read_csv(nov_path)
df['Date'] = pd.to_datetime(df['Date'], format = '%Y-%m-%d')

Mounted at /content/drive/


In [2]:
class Environment:
    def __init__(self, df, initial_money=100000, mode = 'test'):

        self.df = df.dropna().reset_index()

        self.df_total_steps  = len(self.df)-1
        self.initial_money   = initial_money
        self.mode            = mode
        self.trade_time      = None
        self.trade_win       = None
        self.brfore_buy_cash = None
        self.action_space    = np.array([0, 1, 2]) # buy,hold,sell
        self.hold_a_position = None
        self.now_price       = None
        self.cash_in_hand    = None

        self.reset()
        
    def reset(self):

        self.trade_time      = 0
        self.trade_win       = 0
        self.brfore_buy_cash = 0
        self.end_step        = self.df_total_steps
        self.now_step        = 0
        self.hold_a_position = 0.0
        self.now_price       = self.df.loc[self.now_step, 'SP500']
        self.cash_in_hand    = self.initial_money

        return self._get_now_state()

    def step(self, action):

        prev_revenue = self._get_revenue()
        self.now_step += 1
        self.now_price = self.df.loc[self.now_step, 'SP500']
 
        done = (self.end_step == self.now_step)

        self._trade(action,done)
        cur_revenue = self._get_revenue()
 
        reward = cur_revenue - prev_revenue

        if self.mode == 'test':
            info = { 'cur_revenue' : cur_revenue , 'trade_time' : self.trade_time, 'trade_win' : self.trade_win }
        else:
            info = { 'cur_revenue' : cur_revenue }

        return self._get_now_state(), reward, done, info

    def _get_now_state(self):
        state = np.empty(3)
        state[0] = self.hold_a_position
        state[1] = self.now_price
        state[2] = self.cash_in_hand
        return state

    def _get_revenue(self): 
        return self.hold_a_position * self.now_price + self.cash_in_hand

    def _trade(self, action,lastorder = False):
        if lastorder:
            self.cash_in_hand += self.now_price * self.hold_a_position
            self.hold_a_position = 0
            if self.mode == 'test':
                self.trade_time += 1
                if self.cash_in_hand > self.brfore_buy_cash:
                    self.trade_win += 1
        else:
            if self.action_space[0] == action: # buy
                if self.hold_a_position == 0:
                    buy_flag = True
                    if self.mode == 'test':
                        self.brfore_buy_cash = copy.copy(self.cash_in_hand)
                    while buy_flag:
                        if self.cash_in_hand > self.now_price:
                            self.hold_a_position += 1
                            self.cash_in_hand -= self.now_price
                        else:
                            buy_flag = False
            if self.action_space[2] == action: # sell
                if self.hold_a_position != 0:
                    self.cash_in_hand += self.now_price * self.hold_a_position
                    self.hold_a_position = 0
                    if self.mode == 'test':
                        self.trade_time += 1
                        if self.cash_in_hand > self.brfore_buy_cash:
                            self.trade_win += 1

In [3]:
class ReplayMemory:
    def __init__(self, max_size=500, batch_size=32):

        self.cntr = 0
        self.size = 0
        self.max_size = max_size
        self.batch_size = batch_size
        self.states_memory = np.zeros([self.max_size, 3], dtype=np.float32)
        self.next_states_memory = np.zeros([self.max_size, 3], dtype=np.float32)
        self.acts_memory = np.zeros(self.max_size, dtype=np.uint8)
        self.rewards_memory = np.zeros(self.max_size, dtype=np.float32)
        self.done_memory = np.zeros(self.max_size, dtype=np.uint8)

    def store_transition(self, state, act, reward, next_state, done):
        self.states_memory[self.cntr] = state
        self.next_states_memory[self.cntr] = next_state
        self.acts_memory[self.cntr] = act
        self.rewards_memory[self.cntr] = reward
        self.done_memory[self.cntr] = done
        self.cntr = (self.cntr+1) % self.max_size
        self.size = min(self.size+1, self.max_size)

    def random_sampling(self):
        mb_index = np.random.choice(self.size, self.batch_size, replace=False)
        key = ['state','next_state','act','reward','done']
        value = [self.states_memory[mb_index],self.next_states_memory[mb_index],
                 self.acts_memory[mb_index],self.rewards_memory[mb_index],
                 self.done_memory[mb_index]]
        dict1=dict(zip(key,value))

        return dict1

In [4]:
class Brain:
    def __init__(self):

        learning_rate = 0.00001

        input = Input(shape=(3,))
        common = Dense(15, activation='relu')(input)
        common = Dense(15, activation='relu')(common)

        common = Dense(4, activation='linear')(common)
        output = Lambda(lambda a: K.expand_dims(a[:, 0], -1) + a[:, 1:] - 0.0*K.mean(a[:, 1:], keepdims=True),output_shape=(3,))(common)

        model = keras.Model(inputs=input, outputs=output)

        optimizer = Adam(learning_rate=learning_rate)
        model.compile(loss=Huber(), optimizer=optimizer)
        model.summary()
        self.model = model

In [5]:
class Agent(Brain, ReplayMemory):
    def __init__(self, max_size=500, batch_size=32):
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.r = 0.995
        self.batch_size = batch_size
        Brain.__init__(self)
        ReplayMemory.__init__(self, max_size, batch_size)

    def update_replay_memory(self, state, action, reward, next_state, done):
        self.store_transition(state, action, reward, next_state, done)

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(3)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self):
        if self.size < self.batch_size:
            return

        m_batch = self.random_sampling()
        states, next_states, actions, rewards, done = m_batch['state'], m_batch['next_state'], m_batch['act'], m_batch['reward'], m_batch['done']
        target = rewards + (1 - done) * self.gamma * np.amax(self.model.predict(next_states), axis=1)

        target_full = self.model.predict(states)

        target_full[np.arange(self.batch_size), actions] = target
        self.model.train_on_batch(states, target_full)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.r

    def load(self, name):
        self.model.load_weights(name)

    def save(self, name):
        self.model.save_weights(name)

In [6]:
class Main:
    def __init__(self, env, agent, mdl_dir, name, episodes_times = 200, mode = 'test'):
        self.env            = env
        self.agent          = agent
        self.mdl_dir        = mdl_dir
        self.scaler         = self._standard_scaler(self.env)
        self.episodes_times = episodes_times
        self.mode           = mode
        self.name           = name

        if self.mode == 'test':
            self._load()
            self.agent.epsilon = 0.01

            with open(csv_path, 'w') as f:
                row = 'FixedProfit,TradeTimes,TradeWin'
                print(row, file=f)
        else:
            with open(csv_path, 'w') as f:
                row = 'FixedProfit'
                print(row, file=f)

    def play_game(self):

        for episode in range(self.episodes_times):
            state = self.env.reset()
            state = self.scaler.transform([state])
            done  = False
            start_time = datetime.now()
        
            while not done:
                action = self.agent.act(state)
                next_state, reward, done, info = self.env.step(action)
                next_state = self.scaler.transform([next_state])

                if self.mode == 'train':
                    self.agent.update_replay_memory(state, action, reward, next_state, done)
                    self.agent.replay()                
            play_time = datetime.now() - start_time
            if self.mode == 'test':
                print("Episode: {}/{} RapTime: {} FixedProfit: {:.0f} TradeTimes: {} TradeWin: {}".format(episode + 1, episodes_times, play_time, info['cur_revenue'], info['trade_time'], info['trade_win']))
                with open(csv_path, 'a') as f:
                    row = str(info['cur_revenue']) + ',' + str(info['trade_time']) + ',' + str(info['trade_win'])
                    print(row, file=f)
            else:
                print("Episode: {}/{} RapTime: {} FixedProfit: {:.0f}".format(episode + 1, episodes_times, play_time, info['cur_revenue']))
                with open(csv_path, 'a') as f:
                    row = str(info['cur_revenue'])
                    print(row, file=f)
    
            state = next_state

        if self.mode == 'train':
            self._save()

    def _standard_scaler(self, env):
        states = []
        for _ in range(env.df_total_steps):
            action = np.random.choice(env.action_space)
            state, reward, done, info = env.step(action)
            states.append(state)
            if done:
                break
        
        scaler = StandardScaler()
        scaler.fit(states)
        return scaler

    def _load(self):
        with open('{}/{}.pkl'.format(self.mdl_dir, self.name), 'rb') as f:
            self.scaler = pickle.load(f)
        self.agent.load('{}/{}.h5'.format(self.mdl_dir, self.name))


    def _save(self):
        with open('{}/{}.pkl'.format(self.mdl_dir, self.name), 'wb') as f:
            pickle.dump(self.scaler, f)
        self.agent.save('{}/{}.h5'.format(self.mdl_dir, self.name))

In [7]:
initial_money=1000000
episodes_times = 100
batch_size = 32
max_size = 500

env = Environment(df, initial_money=initial_money, mode = mode)
agent = Agent(max_size, batch_size)
main = Main(env, agent, mdl_dir, name, episodes_times, mode)
main.play_game()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense (Dense)                (None, 15)                60        
_________________________________________________________________
dense_1 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 64        
_________________________________________________________________
lambda (Lambda)              (None, 3)                 0         
Total params: 364
Trainable params: 364
Non-trainable params: 0
_________________________________________________________________
Episode: 1/100 RapTime: 0:00:37.301438 FixedProfit: 1000000 TradeTimes: 1 TradeWin: 1
Episode: 2/100 RapTime: 0:00:35.380102 Fixe